<a href="https://colab.research.google.com/github/Hoang-Viet-Tran/BTL_DGHNHT/blob/main/L_HK151_SPE_Silent_Falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project title: Mô phỏng dịch vụ tự phục vụ buffet tại nhiều lối vào đến các khu vực bàn ăn khác nhau
# (Buffet self-service at multiple entrances to different food sections’ tables).

# 1. Team work declaration

## Team: Silent Falcon
| Student               | Id      |
|-----------------------|---------|
| La Anh Tú             | 2112601 |
| Nguyễn Chí Cường      | 2210430 |
| Phạm Anh Hùng         | 2211346 |
| Nguyễn Xuân Hoàng     | 2033528 |
| Nguyễn Công Vũ Hoàng  | 2211087 |
| Đoàn Đức Bình         | 2210302 |
| Lê Võ Trọng Tín       | 2213490 |
| Trần Việt Hoàng       | 2211122 |

## Workmap declaration

| Last name | Student ID | Design Role | Dev Role | Eval Role | Report Role | Ass1 work description (details in text)  | Rating | week 2 outcome | week 3 outcome | week 4 outcome  |
|-----------|------------|-------------|----------|-----------|-------------|------------------------------------------|-----------------|----------------|----------------|-----------------|
| La Anh Tú             | 2112601 | no      | no   | yes    | yes      |                                          | A/B/C           |                |                |                 |
| Nguyễn Chí Cường      | 2210430 | yes      | yes   | no    | yes      |                                          | A/B/C           |                |                |                 |
| Phạm Anh Hùng         | 2211346 | yes      | yes   | no    | yes      |                                          | A/B/C           |                |                |                 |
| Nguyễn Xuân Hoàng     | 2033528 | yes      | yes   | no    | yes      |                                          | A/B/C           |                |                |                 |
| Nguyễn Công Vũ Hoàng  | 2211087 | no      | no   | yes    | yes      |                                          | A/B/C           |                |                |                 |
| Đoàn Đức Bình         | 2210302 | no      | yes   | no    | yes      |                                          | A/B/C           |                |                |                 |
| Lê Võ Trọng Tín       | 2213490 | no      | no   | yes    | yes      |                                          | A/B/C           |                |                |                 |
| Trần Việt Hoàng       | 2211122 | no      | yes   | no    | yes      |                                          | A/B/C           |                |                |                 |



# 2. Thiết kế hệ thống (System design)

## Các bước thực hiện hệ thống (System Performance Steps)

### 1. Xác định mục tiêu và mô tả hệ thống (Define Goal and System)
Mục tiêu của mô phỏng của đề tài là phân tích hiệu suất của hệ thống buffet trong các điều kiện tải (workload) khác nhau, dựa trên các chỉ số như thời gian chờ trung bình, số lượng khách bị từ chối, và số khách được phục vụ.

Hệ thống được mô phỏng là một nhà hàng buffet với các đặc điểm sau:

* **Lối vào (`numEntrance`):** Hệ thống có nhiều lối vào, nơi các nhóm khách hàng đến.
* **Khu vực đồ ăn (`numSection`):** Có nhiều khu vực quầy đồ ăn riêng biệt (ví dụ: khu đồ Nhật, khu đồ Âu, khu tráng miệng).
* **Nhân viên phục vụ/ Server (`m`):** Mỗi khu vực đồ ăn có `m` nhân viên (server) phục vụ. Khách hàng tại mỗi khu được phục vụ song song bởi `m` nhân viên này.
* **Hàng đợi (`K`):** Mỗi khu vực đồ ăn có một hàng đợi giới hạn.

Mô hình này mô phỏng một biến thể của hệ thống hàng đợi **M/M/n/K** (với `n=m`), trong đó thời gian đến và thời gian phục vụ tuân theo phân phối mũ (Exponential).



### 2. Dịch vụ và kết quả (Service and Outcomes)
**Dịch vụ (Service)** : một dịch vụ là quá trình một khách hàng được phục vụ tại một khu đồ ăn bởi một nhân viên (server) phục vụ.

Luồng xử lý của một khách hàng (được định nghĩa trong `beginService`) diễn ra như sau:

1.  **Đến và Chọn khu:** Khách hàng `cid` đến từ lối vào `entrance_id` và chọn ngẫu nhiên khu vực `section_id`.
2.  **Kiểm tra Khu đã chọn (Logic 1):**
    * Hệ thống kiểm tra hàng đợi của `section_id`.
    * Nếu hàng đợi đầy (`len(section.queue) >= self.maxQueueSize` **(K)**), khách hàng *không* vào hàng đợi này.
3.  **Tìm khu thay thế (Logic 2):**
    * Khách hàng quét qua tất cả các khu vực khác để tìm khu vực (`available`) mà hàng đợi chưa đầy.
    * **Nếu tìm thấy:** Khách hàng được chuyển sang khu mới (`available`). Biến `system.relocated` tăng lên.
    * **Nếu không tìm thấy (Tất cả đều đầy):** Khách hàng bị **từ chối** (`system.rejected`) và rời khỏi hệ thống ngay lập tức. Tiến trình kết thúc.
4.  **Vào hàng đợi (Logic 3):**
    * Nếu khu vực ban đầu (hoặc khu vực thay thế) có chỗ, khách hàng yêu cầu một máy chủ (`with section.request() as req`).
    * SimPy tự động đưa khách vào hàng đợi theo cơ chế **FIFO** (First-In, First-Out).
5.  **Phục vụ (Logic 4):**
    * Khi một nhân viên (server) rảnh, khách hàng được phục vụ.
    * Thời gian phục vụ được mô phỏng bằng `random.expovariate(system.serviceRate)`.
6.  **Hoàn tất (Logic 5):**
    * Sau khi hết thời gian phục vụ, khách hàng giải phóng máy chủ.
    * Hàm `endService` được gọi để ghi lại số liệu thống kê.



### 3. Chọn các chỉ số đánh giá (Select Metrics)
Các chỉ số được dùng để đánh giá hệ thống:

*   `waiting_times`: Thời gian chờ trung bình (khoảng thời gian khách chờ trong hàng đợi đến lúc được phục vụ).
*   `service_times`: Thời gian phục vụ trung bình (khoảng thời gian khách được phục vụ)
*   `system_times`: Tổng thời gian khách đến hệ thống và lúc rời đi.
*   `completed`: Số khách hoàn thành phục vụ.
*   `rejected`: Số khách bị từ chối (không còn khu nào trống).
*   `relocated`: Số khách phải chuyển khu (do khu ban đầu đầy).



### 4. Liệt kê các tham số của hệ thống (List System Parameters)
Các tham số của hệ thống:

*   `name`: Tên kịch bản (ví dụ: "Buổi trưa", "Buổi tối").
*   `arrivalRate`: Tốc độ khách đến (khách/phút).
*   `serviceRate`: Tốc độ phục vụ (khách/phút) *cho mỗi server*.
*   `m`: Số nhân viên (server) mỗi khu.
*   `K`: Dung lượng hàng đợi.
*   `duration`: Tổng thời gian mô phỏng (phút).
*   `groupMin`, `groupMax`: Kích thước nhóm khách.



### 5. Các yếu tố khảo sát (List Factors to Study)


### 6. Kỹ thuật đánh giá (Evaluation Technique)
Hệ thống được đánh giá thông qua:

*   Mô phỏng sự kiện rời rạc **(Discrete-event simulation)**, dùng thư việ SimPy.
*   Mỗi sự kiện bao gồm: khách đến, khách vào hàng đợi, bắt đầu phục vụ và kết thúc phục vụ.
*   Các thời gian đến và phục vụ được sinh bằng phân phối mũ.
*   Mỗi kịch bản sẽ chạy độc lập với cấu hình riêng.
*   Sau khi chạy xong, thống kê hệ thống được tổng hợp để so sánh.



### 7. Chọn Workload (Select Workload)
Hai tải điển hình có thể được lựa chọn:
*   Thí nghiệm 1 - Tải vừa (Moderate traffic): lượng khách vừa phải, cấu hình hệ thống ổn định.
*   Thí nghiệm 2 - Tải cao (Heavy traffic - Giờ cao điểm): lượng khách lớn, khả năng gây tắc nghẽn, nhiều người bị từ chối hoặc chuyển khu.

Sử dụng các `scenario` để thử nghiệm.



## System Component/Module description

# 3. Hiện thực hệ thống (System Implementation)



Install prerequisite library


*   simpy



In [ ]:
pip install simpy

## Queuing Node 1: *arrival + log_event + schedule*

In [ ]:
import simpy
import random

def log_event(message, timestamp):
    print(f"[t={timestamp:.2f} phút] {message}")

def schedule(system, cid, entrance_id, section_id):
    """
    Lên lịch phục vụ khách — tạo process beginService().
    """
    system.env.process(beginService(system, cid, entrance_id, section_id))

def arrival(system, entrance_id):
    """
    Xử lý sự kiện khách đến theo nhóm.
    Mỗi nhóm có số lượng ngẫu nhiên trong [groupMin, groupMax].
    """

    while True:
        inter_arrival = random.expovariate(system.arrivalRate)
        yield system.env.timeout(inter_arrival)

        group_size = random.randint(system.groupMin, system.groupMax)
        log_event(f"👥 {group_size} khách đến cùng lúc tại lối {entrance_id}", system.env.now)

        for _ in range(group_size):
            cid = system.next_cid
            system.next_cid += 1

            section_id = random.randint(0, system.numSection - 1)
            log_event(f"   ↳ Khách {cid} chọn khu {section_id}", system.env.now)
            schedule(system, cid, entrance_id, section_id)


## Queuing node 2: *Service logic: beginService + endService*


In [ ]:
# === beginService() ===
def beginService(system, cid, entrance_id, section_id):
    """
    Bắt đầu phục vụ khách (nếu có server rảnh), hoặc chờ trong hàng đợi.
    Nếu hàng đợi khu này đầy, khách sẽ chuyển sang khu khác.
    Nếu toàn hệ thống đều đầy, khách bị từ chối.
    """

    arrival_time = system.env.now
    sections = system.sections

    # --- 1. kiểm tra khu đã chọn có chỗ không ---
    section = sections[section_id]
    if len(section.queue) >= system.maxQueueSize:  # QueueSize
        # thử tìm khu khác còn trống
        available = None
        for alt_id, alt_section in enumerate(sections):
            if len(alt_section.queue) < system.maxQueueSize:
                available = alt_id
                break

        # --- 2. nếu không có khu nào trống ---
        if available is None:
            system.rejected += 1
            log_event(f"❌ Khách {cid} (lối {entrance_id}) bị từ chối — tất cả khu đều đầy.", system.env.now)
            return
        else:
            system.relocated += 1
            log_event(f"↪️ Khách {cid} (lối {entrance_id}) chuyển từ khu {section_id} sang khu {available} (khu cũ đầy).", system.env.now)
            section_id = available
            section = sections[section_id]

    # --- 3. đăng ký vào hàng đợi khu mới (FIFO) ---
    with section.request() as req:
        yield req  # khách phải đợi đến lượt (SimPy xử lý FIFO)

        # Thời điểm bắt đầu được phục vụ
        service_start = system.env.now
        waiting_time = service_start - arrival_time

        # --- 4. Khi đến lượt phục vụ ---
        service_time = random.expovariate(system.serviceRate)
        yield system.env.timeout(service_time)

        # --- 5. Hoàn tất phục vụ ---
        endService(system, cid, entrance_id, section_id, arrival_time, waiting_time, service_time)


# === endService() ===
def endService(system, cid, entrance_id, section_id, arrival_time, waiting_time, service_time):
    """
    Kết thúc quá trình phục vụ khách.
    """

    end_time = system.env.now
    total_time = end_time - arrival_time

    system.completed += 1

    system.waiting_times.append(waiting_time)
    system.service_times.append(service_time)
    system.system_times.append(total_time)

    log_event(
        f"✅ Khách {cid} (lối {entrance_id}) rời khu {section_id} sau {total_time:.2f} phút "
        f"(chờ {waiting_time:.2f} phút, phục vụ {service_time:.2f} phút).", system.env.now
    )

## Queuing node 3: *System class + summarize_results*


In [ ]:
class BuffetSys:
    def __init__(self, env, arrivalRate, serviceRate, numEntrance, numSection,
                 m, K, D='FIFO', groupMin=2, groupMax=5):

        self.env = env
        self.arrivalRate = arrivalRate
        self.serviceRate = serviceRate
        self.numEntrance = numEntrance
        self.numSection = numSection
        self.maxServer = m
        self.maxQueueSize = K
        self.discipline = D
        self.groupMin = groupMin
        self.groupMax = groupMax

        # Tạo n server cho mỗi khu vực món ăn
        self.sections = [
            simpy.Resource(env, capacity=self.maxServer) for _ in range(numSection)
        ]

        # Biến thống kê
        self.next_cid = 0

        self.completed = 0

        self.waiting_times = []     # Wq: Thời gian chờ trong hàng
        self.service_times = []     # S: Thời gian phục vụ
        self.system_times = []      # W: tổng thời gian trong hệ thống

        self.rejected = 0       # khách bị từ chối
        self.relocated = 0      # khách đổi khu

    def init(self):
        """Khởi tạo các tiến trình arrival ban đầu"""
        for i in range(self.numEntrance):
            self.env.process(arrival(self, entrance_id=i))

    def run(self, duration):
        self.init()
        log_event("Bắt đầu mô phỏng buffet M/M/n...", self.env.now)
        self.env.run(until=duration)
        log_event("Kết thúc mô phỏng.", self.env.now)


def summarize_results(system):
    print("\n=== Thống kê mô phỏng ===")
    print(f"• Số khách phục vụ xong: {system.completed}")
    print(f"• Số khách bị từ chối:   {system.rejected}")
    print(f"• Số khách đổi khu:      {system.relocated}")

    if system.completed > 0:
        avg_waiting = sum(system.waiting_times) / len(system.waiting_times) # E[Wq]
        avg_service = sum(system.service_times) / len(system.service_times) # E[S]
        avg_system  = sum(system.system_times)  / len(system.system_times)  # E[W]

        print(f"• Thời gian chờ trung bình:             {avg_waiting:.2f} phút")
        print(f"• Thời gian phục vụ trung bình:         {avg_service:.2f} phút")
        print(f"• Thời gian trong hệ thống trung bình:  {avg_system:.2f} phút")
    else:
        print("• Không có khách nào được phục vụ.")
    print("==============================\n")

## System configuration and execution

In [ ]:
"""
Mô phỏng hệ thống buffet M/M/n theo đơn vị PHÚT
------------------------------------------------
arrivalRate: Tốc độ khách đến trung bình (khách/phút)
serviceRate: Tốc độ phục vụ trung bình của mỗi server (khách/phút)
numEntrance: số lối vào buffet
numSection: số khu vực đồ ăn
m: số nhân viên phục vụ mỗi khu
K: dung lượng tối đa của hàng đợi
duration: tổng thời gian mô phỏng (phút)
groupMin/groupMax = số khách tối thiểu / tối đa trong một nhóm đến cùng lúc
"""

print(" Môi trường mô phỏng Buffet M/M/n/K sẵn sàng! ")

scenario = [
    {
        "name": "Thí nghiệm 1 - Moderate traffic",
        "arrivalRate": 5,
        "serviceRate": 8,
        "m": 2,
        "K": 3,
        "duration": 30,
        "groupMin": 1,
        "groupMax": 4
    },
    {
        "name": "Thí nghiệm 2 - Dinner time",
        "arrivalRate": 12,
        "serviceRate": 10,
        "m": 10,
        "K": 3,
        "duration": 30,
        "groupMin": 1,
        "groupMax": 16
    }
]


# --- Hiển thị danh sách kịch bản ---
print("\nDanh sách kịch bản:")
for i, sc in enumerate(scenario, 1):
    print(f"{i}: {sc['name']}")

# --- Người dùng chọn kịch bản ---
choice = int(input("\nNhập số kịch bản bạn muốn chạy (1 hoặc 2): ")) - 1

if choice not in range(len(scenario)):
    print("Lựa chọn không hợp lệ! Chạy kịch bản mặc định 1.")
    choice = 0

sc = scenario[choice]

print(f"\n==== Chạy kịch bản {choice+1}: {sc['name']} ====")

# --- Khởi tạo môi trường và hệ thống ---
env = simpy.Environment()
buffet = BuffetSys(
    env,
    arrivalRate=sc["arrivalRate"],
    serviceRate=sc["serviceRate"],
    numEntrance=3,
    numSection=4,
    m=sc["m"],
    K=sc["K"],
    D="FIFO",
    groupMin=sc["groupMin"],
    groupMax=sc["groupMax"]
)

# --- Chạy mô phỏng ---
buffet.run(duration=sc["duration"])

    # Thống kê kết quả
summarize_results(buffet)


# 4. Trình bày kết quả (Present results)

In [ ]:
=== Thống kê mô phỏng ===
• Số khách phục vụ xong: 1060
• Số khách bị từ chối:   7
• Số khách đổi khu:      69
• Thời gian chờ trung bình:             0.05 phút
• Thời gian phục vụ trung bình:         0.12 phút
• Thời gian trong hệ thống trung bình:  0.17 phút
==============================

Experiment 1 - Tải trung bình

Kết quả sau mô phỏng:


*   Ít khách bị từ chối
*   Thời gian chờ thấp
*   Hệ thống hoạt động ổn định

Nhận xét:

Hệ thống hoạt động ổn định trong điều kiện tải trung bình.

Các quầy phục vụ đủ nhanh để hạn chế hình thành hàng dài, và phân bổ khách diễn ra khá đều.

Đây là trạng thái “lý tưởng” của hệ thống buffet trong thực tế



In [ ]:
=== Thống kê mô phỏng ===
• Số khách phục vụ xong: 8431
• Số khách bị từ chối:   942
• Số khách đổi khu:      1069
• Thời gian chờ trung bình:             0.01 phút
• Thời gian phục vụ trung bình:         0.10 phút
• Thời gian trong hệ thống trung bình:  0.11 phút

Experiment 2 - Tải cao (giờ cao điểm)

Kết quả sau mô phỏng:


*   Nhiều khách bị từ chối
*   Nhiều khách phải chuyển khu
*   Thể hiện tình trạng quá tải

Nhận xét:

Hệ thống quá tải trong phần lớn thời gian mô phỏng.

Dù số lượng server tại mỗi khu có thể giúp tăng throughput, nhưng dung lượng hàng đợi hạn chế (K nhỏ) khiến nhiều khách phải rời đi.

Đây là tình huống tương tự giờ cao điểm tại buffet: khách đông nhưng không thể phục vụ hết, dẫn đến ùn tắc và giảm trải nghiệm người dùng



# 5. Kết luận (Conclusions)

Project coordinator provide the statements of the work's including the superior point and 2-3 limitation provided by the testing member.

Trong dự án này, nhóm đã xây dựng mô hình mô phỏng hệ thống buffet tự phục vụ dựa trên mô hình hàng đợi M/M/n/K bằng thư viện SimPy.

Hệ thống tái hiện nhiều yếu tố thực tế như khách đến theo nhóm, nhiều lối vào, nhiều khu đồ ăn hoạt động song song, dung lượng hàng đợi có hạn và logic chuyển khu khi quá tải.

Điểm mạnh:


1.   Cấu trúc mô hình rõ ràng, bám sát lý thuyết hàng đợi M/M/n/K
2.   Hỗ trợ nhiều kịch bản mô phỏng khác nhau
3.   Ghi lại đầy đủ các chỉ số hiệu năng của hệ thống


Hạn chế:


1.   Khách chọn khu hoàn toàn ngẫu nhiên
2.   Giả định thời gian đến và phục vụ là phân phối mũ – có thể chưa hoàn toàn giống thực tế



